# 12장 내장 자료형 상속과 다중 상속

이 장에서는 상속에 대해 설명하며 특히 다음 두 가지 특징을 집중적으로 다룬다.

* 내장 자료형 상속의 위험성
* 다중 상속과 메서드 결정 순서

## 12.1 내장 자료형의 상속은 까다롭다

C 언어로 작성된 내장 클래스의 코드는 사용자가 오버라이드한 코드를 호출하지 않으므로 주의가 필요하다.

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2) 

In [5]:
dd = DoppelDict(one=1) # dict 클래스의 __init__() 메서드는 __setitem__() 이 오버라이드 되었다는 사실을 무시
dd

{'one': 1}

In [6]:
dd['two']=2 # 오버라이드된 __setitem__()을 호출하므로 [2,2]에 매핑
dd

{'one': 1, 'two': [2, 2]}

In [7]:
dd.update(three=3) # update() 메서드는 오버라이드된 __setitem__() 메서드를 호출하지 않는다
dd

{'one': 1, 'two': [2, 2], 'three': 3}

이 문제는 `self.get()`이 `self.__getitem__()`을 호출하는 경우처럼 객체 안에서 호출할 때뿐만 아니라, 내장 메서드가 호출하는 다른 클래스의 오버라이드된 메서드에서도 발생한다.

In [8]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [12]:
ad = AnswerDict(a='foo') # ad는 ('a','foo') 키-값 쌍으로 채운 AnswerDict 객체
ad['a'] # __getitem__() 은 42를 반환

42

In [13]:
d = {}
d.update(ad) # update() 메서드는 AnswerDict.__getitem__() 메서드를 무시
d['a'] 

'foo'

In [14]:
d

{'a': 'foo'}

`dict`, `list`, `str` 등의 내장 자료형은 사용자가 정의한 오버라이드된 메서드를 무시하므로, 내장 자료형보다는 `UserDict`, `UserList`, `UserString` 등을 사용하는 것이 좋다.

In [15]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

In [16]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [17]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [18]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [19]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [20]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [21]:
d = {}
d.update(ad) 
d['a'] 

42

In [22]:
d

{'a': 42}

문제 없다!

## 12.2 다중 상속과 메서드 결정 순서

다중 상속을 지원하는 언어에서는 별개의 상위클래스가 동일한 이름으로 메서드를 구현할 때 발생하는 이름 충돌 문제를 해결해야 한다.

In [32]:
# diamond.py

class A:
    def ping(self):
        print('ping:', self)
        
class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [33]:
from diamond import *
d = D()
d.pong()

pong: <diamond.D object at 0x7f1c4bc3b630>


In [34]:
C.pong(d)

PONG: <diamond.D object at 0x7f1c4bc3b630>


파이썬이 상속 그래프를 조회할 때는 `메서드 결정 순서(MRO)`라고 하는 특정한 순서를 따른다.

In [35]:
D.__mro__

(diamond.D, diamond.B, diamond.C, diamond.A, object)

슈퍼클래스 메서드에 위임할 때는 내장 함수인 `super()`를 사용하는 것이 좋다. 그러나 MRO를 우회해서 슈퍼클래스 메서드를 직접 호출할 수도 있다. 예를 들어 `D.ping()` 메서드는 다음과 같이 구현할 수 있다.

```python
    def ping(self):
        A.ping(self) # super().ping() 대신 호출
        print('post-ping:', self)
```

객체 메서드를 클래스에 직접 호출할 때는 바인딩되지 않은 메서드에 접근하는 것이므로 `self`를 반드시 명시해야 한다.

In [36]:
d.ping()

ping: <diamond.D object at 0x7f1c4bc3b630>
post-ping: <diamond.D object at 0x7f1c4bc3b630>


위 명령은 `A.ping()`과 `print('post-ping:', self)`가 각각 출력한 것이다.

In [37]:
d.pingpong()

ping: <diamond.D object at 0x7f1c4bc3b630>
post-ping: <diamond.D object at 0x7f1c4bc3b630>
ping: <diamond.D object at 0x7f1c4bc3b630>
pong: <diamond.D object at 0x7f1c4bc3b630>
pong: <diamond.D object at 0x7f1c4bc3b630>
PONG: <diamond.D object at 0x7f1c4bc3b630>


위 명령 중 `self.pong()`과 `super().pong()`은 `__mro__`에 따라 `B.pong()`을 호출한다. 다만 `C.pong(self)`는 `__mro__`를 무시하고 `C.pong()`을 호출한다. 만일 diamond.py에서 D 클래스를 `class D(C, B)`로 선언했다면 B보다 C를 먼저 찾도록 MRO가 변경된다. 

다음은 여러 익숙한 클래스의 `__mro__`를 조사한 것이다.

In [6]:
bool.__mro__

(bool, int, object)

In [9]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [10]:
print_mro(bool)

bool, int, object


In [12]:
!cat frenchdeck2.py

import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    # MutableSequence를 상속하므로 이 클래스의 추상 메서드인 
    # __delitem__() 도 구현해야함
    def __delitem__(self, position):
        del self._cards[position]
        
    # insert() 또한 추상 메서드
    def insert(self, position, value):
        self._cards.insert(position, value)

In [11]:
from frenchdeck2 import FrenchDeck2
print_mro(FrenchDeck2)

FrenchDeck2, MutableSequence, Sequence, Reversible, Collection, Sized, Iterable, Container, object


In [13]:
import numbers
print_mro(numbers.Integral)

Integral, Rational, Real, Complex, Number, object


In [14]:
import io
print_mro(io.BytesIO)

BytesIO, _BufferedIOBase, _IOBase, object


In [15]:
print_mro(io.TextIOWrapper)

TextIOWrapper, _TextIOBase, _IOBase, object


`open()`으로 파일을 열 때 이진 파일의 경우 `BytesIO`, 텍스트 파일의 경우 `TextIOWrapper` 객체가 반환된다.

## 12.3 실세계에서의 다중 상속

이 이후는 안해도 될듯...